Original:

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/3_regularization.ipynb

Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_labels = save['test_labels']
del save  # hint to help gc free up memory
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28

num_features = image_size * image_size
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, num_features)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [19]:
def logistic_regression_l2(input_size,
                           num_classes,
                           learning_rate=0.1,
                           l2_coef=0.1,
                           use_l2=True):
    X = tf.placeholder(tf.float32, shape=(None, input_size))
    y = tf.placeholder(tf.float32, shape=(None, num_classes))

    weights = tf.Variable(
        tf.truncated_normal([input_size, num_classes]))
    biases = tf.Variable(tf.zeros([num_classes]))

    logits = tf.matmul(X, weights) + biases
    prediction = tf.nn.softmax(logits)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    if use_l2:
        loss += l2_coef * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    return X, y, prediction, loss, optimizer


def feedforward_l2_classifier(input_size,
                              num_classes,
                              hidden_size,
                              learning_rate=0.01,
                              l2_coef=0.01,
                              use_l2=True):
    X = tf.placeholder(tf.float32, shape=(None, input_size))
    y = tf.placeholder(tf.float32, shape=(None, num_classes))

    hidden_weights = tf.Variable(
        tf.truncated_normal([input_size, hidden_size]))
    hidden_biases = tf.Variable(tf.zeros([hidden_size]))

    hidden_layer = tf.nn.relu(tf.matmul(X, hidden_weights) + hidden_biases)
    
    output_weights = tf.Variable(
        tf.truncated_normal([hidden_size, num_classes]))
    output_biases = tf.Variable(tf.zeros([num_classes]))
    
    logits = tf.matmul(hidden_layer, output_weights) + output_biases
    prediction = tf.nn.softmax(logits)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    if use_l2:
        loss += l2_coef * tf.nn.l2_loss(hidden_weights)
        loss += l2_coef * tf.nn.l2_loss(output_weights)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    return X, y, prediction, loss, optimizer


def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

def train(model_fn,
          num_epochs,
          batch_size,
          train_dataset=train_dataset,
          train_labels=train_labels,
          shuffle_epoch=False):
    
    epoch_size = train_labels.shape[0]
    epoch_size -= epoch_size % batch_size
    num_steps = epoch_size // batch_size
        
    with tf.Graph().as_default() as graph, \
        tf.Session(graph=graph) as session:

        X, y, prediction, loss_op, opt = model_fn()

        tf.global_variables_initializer().run()
        print('Initialized.\n')

        for epoch in range(num_epochs):
            print('Epoch {}\n'.format(epoch+1))
            
            if shuffle_epoch and epoch:
                print('Shuffling...\n')
                train_dataset, train_labels = randomize(train_dataset, train_labels)
            
            avg_loss = 0
            avg_acc = 0
            for step in range(num_steps):
                offset = step * batch_size
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]

                feed_dict = {X: batch_data, y: batch_labels}
                _, loss, predictions = session.run(
                    [opt, loss_op, prediction], feed_dict=feed_dict)
                
                avg_loss = (loss + step * avg_loss) / (step + 1)
                
                acc = accuracy(predictions, batch_labels)
                avg_acc = (acc + step * avg_acc) / (step + 1)

            print('Avarage loss: {:.3f}'.format(avg_loss))
            print('Avarage accuracy: {:.1f}%'.format(avg_acc))
 
            valid_pred = session.run(prediction, feed_dict={X: valid_dataset}) 
            valid_acc = accuracy(valid_pred, valid_labels)
            print('Validation accuracy: {:.1f}%\n'.format(valid_acc))

        test_pred = session.run(prediction, feed_dict={X: test_dataset})
        test_acc = accuracy(test_pred, test_labels)
        print('\nTest accuracy: {:.2f}%\n'.format(test_acc))

In [20]:
%%time

train(lambda: logistic_regression_l2(num_features, num_labels),
      num_epochs=5,
      batch_size=100)

Initialized.

Epoch 1

Avarage loss: 8.598
Avarage accuracy: 77.9%
Validation accuracy: 80.1%

Epoch 2

Avarage loss: 1.009
Avarage accuracy: 80.5%
Validation accuracy: 80.2%

Epoch 3

Avarage loss: 1.009
Avarage accuracy: 80.5%
Validation accuracy: 80.2%

Epoch 4

Avarage loss: 1.009
Avarage accuracy: 80.5%
Validation accuracy: 80.2%

Epoch 5

Avarage loss: 1.009
Avarage accuracy: 80.5%
Validation accuracy: 80.2%


Test accuracy: 87.57%

CPU times: user 47.5 s, sys: 2.68 s, total: 50.2 s
Wall time: 22.4 s


In [21]:
%%time

train(lambda: feedforward_l2_classifier(num_features, num_labels, 1024),
      num_epochs=5,
      batch_size=100)

Initialized.

Epoch 1

Avarage loss: 2614.717
Avarage accuracy: 75.9%
Validation accuracy: 78.9%

Epoch 2

Avarage loss: 1739.673
Avarage accuracy: 80.3%
Validation accuracy: 80.2%

Epoch 3

Avarage loss: 1162.244
Avarage accuracy: 81.9%
Validation accuracy: 81.3%

Epoch 4

Avarage loss: 777.222
Avarage accuracy: 82.9%
Validation accuracy: 82.1%

Epoch 5

Avarage loss: 520.029
Avarage accuracy: 83.8%
Validation accuracy: 82.7%


Test accuracy: 89.68%

CPU times: user 17min 5s, sys: 58.2 s, total: 18min 3s
Wall time: 3min 3s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
batch_size = 100
num_batches = 5
num_examples = num_batches * batch_size

tiny_dataset = train_dataset[:num_examples]
tiny_labels = train_labels[:num_examples]

print('Tiny dataset:', tiny_dataset.shape)
print('Tiny labels:', tiny_labels.shape)

Tiny dataset: (500, 784)
Tiny labels: (500, 10)


In [24]:
%%time

train(lambda: logistic_regression_l2(num_features, num_labels),
      num_epochs=5,
      batch_size=batch_size,
      train_dataset=tiny_dataset,
      train_labels=tiny_labels)

Initialized.

Epoch 1

Avarage loss: 308.298
Avarage accuracy: 6.4%
Validation accuracy: 7.2%

Epoch 2

Avarage loss: 275.671
Avarage accuracy: 9.0%
Validation accuracy: 10.0%

Epoch 3

Avarage loss: 246.968
Avarage accuracy: 13.6%
Validation accuracy: 15.2%

Epoch 4

Avarage loss: 221.646
Avarage accuracy: 19.0%
Validation accuracy: 19.7%

Epoch 5

Avarage loss: 199.191
Avarage accuracy: 26.8%
Validation accuracy: 24.1%


Test accuracy: 26.66%

CPU times: user 640 ms, sys: 8 ms, total: 648 ms
Wall time: 245 ms


In [25]:
%%time

train(lambda: feedforward_l2_classifier(num_features, num_labels, 1024),
      num_epochs=5,
      batch_size=batch_size,
      train_dataset=tiny_dataset,
      train_labels=tiny_labels)

Initialized.

Epoch 1

Avarage loss: 3468.046
Avarage accuracy: 7.4%
Validation accuracy: 13.6%

Epoch 2

Avarage loss: 3328.323
Avarage accuracy: 20.6%
Validation accuracy: 24.0%

Epoch 3

Avarage loss: 3266.453
Avarage accuracy: 33.0%
Validation accuracy: 31.6%

Epoch 4

Avarage loss: 3231.950
Avarage accuracy: 41.2%
Validation accuracy: 39.1%

Epoch 5

Avarage loss: 3209.292
Avarage accuracy: 49.6%
Validation accuracy: 43.8%


Test accuracy: 49.50%

CPU times: user 20.6 s, sys: 272 ms, total: 20.9 s
Wall time: 2.99 s


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [26]:
%%time

def feedforward_dropout_classifier(input_size,
                                   num_classes,
                                   hidden_size,
                                   learning_rate=0.01,
                                   dropout_keep_prob=0.5,
                                   use_dropout=True):
    X = tf.placeholder(tf.float32, shape=(None, input_size))
    y = tf.placeholder(tf.float32, shape=(None, num_classes))

    hidden_weights = tf.Variable(
        tf.truncated_normal([input_size, hidden_size]))
    hidden_biases = tf.Variable(tf.zeros([hidden_size]))

    hidden_layer = tf.nn.relu(tf.matmul(X, hidden_weights) + hidden_biases)
    
    output_weights = tf.Variable(
        tf.truncated_normal([hidden_size, num_classes]))
    output_biases = tf.Variable(tf.zeros([num_classes]))
    
    logits = tf.matmul(hidden_layer, output_weights) + output_biases
    prediction = tf.nn.softmax(logits)

    if use_dropout:
        # rewire logits using dropout (will not change prediction path)
        hidden_layer = tf.nn.dropout(hidden_layer, dropout_keep_prob)
        logits = tf.matmul(hidden_layer, output_weights) + output_biases

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    return X, y, prediction, loss, optimizer

train(lambda: feedforward_dropout_classifier(num_features, num_labels, 1024),
      num_epochs=5,
      batch_size=batch_size)

Initialized.

Epoch 1

Avarage loss: 75.115
Avarage accuracy: 80.4%
Validation accuracy: 82.8%

Epoch 2

Avarage loss: 37.079
Avarage accuracy: 83.6%
Validation accuracy: 83.9%

Epoch 3

Avarage loss: 24.460
Avarage accuracy: 84.4%
Validation accuracy: 84.0%

Epoch 4

Avarage loss: 17.364
Avarage accuracy: 84.9%
Validation accuracy: 84.3%

Epoch 5

Avarage loss: 12.556
Avarage accuracy: 85.1%
Validation accuracy: 84.2%


Test accuracy: 90.81%

CPU times: user 16min 40s, sys: 38.6 s, total: 17min 19s
Wall time: 2min 56s


In [27]:
%%time

train(lambda: feedforward_dropout_classifier(num_features, num_labels, 1024),
      num_epochs=5,
      batch_size=batch_size,
      train_dataset=tiny_dataset,
      train_labels=tiny_labels)

Initialized.

Epoch 1

Avarage loss: 467.770
Avarage accuracy: 10.2%
Validation accuracy: 18.1%

Epoch 2

Avarage loss: 340.579
Avarage accuracy: 26.8%
Validation accuracy: 32.4%

Epoch 3

Avarage loss: 286.574
Avarage accuracy: 41.4%
Validation accuracy: 42.1%

Epoch 4

Avarage loss: 240.305
Avarage accuracy: 50.2%
Validation accuracy: 47.9%

Epoch 5

Avarage loss: 198.983
Avarage accuracy: 58.0%
Validation accuracy: 53.3%


Test accuracy: 59.73%

CPU times: user 20.5 s, sys: 244 ms, total: 20.8 s
Wall time: 2.95 s


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [52]:
%%time

def layer_with_dropout(X, input_size, num_units, dropout_X=None, dropout_keep_prob=1.0):
    W = tf.Variable(
        tf.truncated_normal(shape=(input_size, num_units), stddev=1.0 / np.sqrt(input_size)))
    b = tf.Variable(tf.zeros(shape=(num_units,)))
    layer_ = tf.nn.relu_layer(X, W, b)
    dropout_ = None
    if dropout_X is not None:
        dropout_ = tf.nn.relu_layer(dropout_X, W, b)
        dropout_ = tf.nn.dropout(dropout_, dropout_keep_prob)
    return layer_, dropout_

def feedforward_multilayer_classifier(input_size,
                                      num_classes,
                                      hidden_sizes,
                                      dropout_keep_prob=0.5,
                                      use_dropout=True):

    X = tf.placeholder(tf.float32, shape=(None, input_size))
    y = tf.placeholder(tf.float32, shape=(None, num_classes))

    print('Input:\n')
    print(X, '\n')
        
    hidden_sizes = [input_size] + hidden_sizes
    hidden_layer = X
    dropout_layer = X if use_dropout else None
    for k in range(1, len(hidden_sizes)):
        hidden_layer, dropout_layer = layer_with_dropout(
            hidden_layer, hidden_sizes[k-1], hidden_sizes[k], dropout_layer, dropout_keep_prob)
        print('Hidden Layer {}:\n'.format(k))
        print(hidden_layer)
        if use_dropout:
            print(dropout_layer)
        print()
    
    output_W = tf.Variable(tf.truncated_normal(shape=(hidden_sizes[-1], num_classes)))
    output_b = tf.Variable(tf.zeros(shape=(num_classes,)))
    
    logits = tf.nn.xw_plus_b(hidden_layer, output_W, output_b)
    prediction = tf.nn.softmax(logits)

    if use_dropout:
        # rewire logits using dropout (will not change prediction path)
        logits = tf.nn.xw_plus_b(dropout_layer, output_W, output_b)

    print('Output (logits):\n')
    print(logits, '\n')

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

    return X, y, prediction, loss

def model_opt_decay(model_fn,
              starter_learning_rate=0.001,
              decay_steps=5_000,
              decay_rate=0.98,
              staircase=True):    
    X, y, prediction, loss = model_fn()
    
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(
        starter_learning_rate, global_step, decay_steps, decay_rate, staircase=staircase)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    return X, y, prediction, loss, optimizer

def model_opt_adam(model_fn, learning_rate=0.001, clip_grads=1):
    X, y, prediction, loss = model_fn()

    opt_fn = tf.train.AdamOptimizer(learning_rate)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), clip_grads)
    optimizer = opt_fn.apply_gradients(zip(grads, tvars))
    
    return X, y, prediction, loss, optimizer


model_fn = lambda: feedforward_multilayer_classifier(num_features,
                                                     num_labels,
                                                     [128, 64, 32],
                                                     use_dropout=True)

model_opt = lambda: model_opt_adam(model_fn,
                                   learning_rate=0.001)

train(model_opt,
      num_epochs=50,
      batch_size=100,
      shuffle_epoch=True)

Input:

Tensor("Placeholder:0", shape=(?, 784), dtype=float32) 

Hidden Layer 1:

Tensor("relu_layer:0", shape=(?, 128), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 128), dtype=float32)

Hidden Layer 2:

Tensor("relu_layer_2:0", shape=(?, 64), dtype=float32)
Tensor("dropout_1/mul:0", shape=(?, 64), dtype=float32)

Hidden Layer 3:

Tensor("relu_layer_4:0", shape=(?, 32), dtype=float32)
Tensor("dropout_2/mul:0", shape=(?, 32), dtype=float32)

Output (logits):

Tensor("xw_plus_b_1:0", shape=(?, 10), dtype=float32) 

Initialized.

Epoch 1

Avarage loss: 1.053
Avarage accuracy: 79.8%
Validation accuracy: 82.9%

Epoch 2

Shuffling...

Avarage loss: 0.767
Avarage accuracy: 83.7%
Validation accuracy: 83.6%

Epoch 3

Shuffling...

Avarage loss: 0.719
Avarage accuracy: 84.3%
Validation accuracy: 84.5%

Epoch 4

Shuffling...

Avarage loss: 0.692
Avarage accuracy: 84.8%
Validation accuracy: 84.3%

Epoch 5

Shuffling...

Avarage loss: 0.679
Avarage accuracy: 85.0%
Validation accuracy: 84.5%

E